<a href="https://colab.research.google.com/github/Decoding-Data-Science/aiguild/blob/main/Copy_of_DDS_Academy_LLM_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DDS Academy – Feedback Sentiment Analysis (LLM Prompting)

This notebook demonstrates **LLM-based sentiment + tone classification** using prompt engineering (no training) for DDS Academy / AI Residency feedback.

**Labels:** positive, negative, sarcastic, neutral

## Step 1: Load OpenAI API Key from Colab Secrets

get ypur APi keys https://platform.openai.com/api-keys

In [1]:
from google.colab import userdata
import os

api_key = userdata.get('openai')
if not api_key:
    raise ValueError('OPENAI_API_KEY not found in Colab Secrets')

os.environ['OPENAI_API_KEY'] = api_key
print('✅ OpenAI API key loaded')

✅ OpenAI API key loaded


## Step 2: Install & Import Libraries

In [2]:
!pip -q install --upgrade openai pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [3]:
import json
import pandas as pd
from openai import OpenAI
from sklearn.metrics import accuracy_score, classification_report

client = OpenAI()

## Step 3: Sample DDS Academy Feedback

In [4]:
data = [
    {'text': 'Loved the AI Residency session. Clear examples and very practical.', 'gold': 'positive'},
    {'text': 'One of the best AI workshops I have attended.', 'gold': 'positive'},
    {'text': "The mentor's explanation was crystal clear, and the practical exercises were very helpful.", 'gold': 'positive'},
    {'text': 'Excellent content and delivery. Highly recommended for anyone interested in AI.', 'gold': 'positive'},
    {'text': 'Too fast. Hard to follow the LLM and RAG parts.', 'gold': 'negative'},
    {'text': 'Audio issues made it difficult to concentrate.', 'gold': 'negative'},
    {'text': 'The pace was too slow for experienced developers, and some topics were superficial.', 'gold': 'negative'},
    {'text': 'Disappointed with the lack of in-depth technical discussion; it felt more like an overview.', 'gold': 'negative'},
    {'text': "Wow amazing… another it depends answer. Super helpful 🙃", 'gold': 'sarcastic'},
    {'text': 'Sure, production deployment is that easy. Totally.', 'gold': 'sarcastic'},
    {'text': "Oh, a 3-hour lecture on 'hello world' is exactly what I signed up for.", 'gold': 'sarcastic'},
    {'text': 'The hands-on part was so hands-on, I barely touched the keyboard.', 'gold': 'sarcastic'},
    {'text': 'Session covered prompt engineering basics.', 'gold': 'neutral'},
    {'text': 'Joined late, missed the introduction.', 'gold': 'neutral'},
    {'text': 'The presentation slides were mostly text and bullet points.', 'gold': 'neutral'},
    {'text': 'The workshop started at 9 AM and ended at 5 PM.', 'gold': 'neutral'},
    {'text': 'This session was surprisingly good, unlike the last one.', 'gold': 'positive'}, # Edge case: subtle comparison
    {'text': 'I found it challenging but ultimately rewarding.', 'gold': 'positive'}, # Edge case: mixed sentiment leading to positive
    {'text': 'The content was not bad, I guess.', 'gold': 'neutral'}, # Edge case: understated positive/neutral
    {'text': 'It was good, I suppose, if you like that sort of thing.', 'gold': 'sarcastic'}, # Edge case: subtle sarcasm
    {'text': 'The speaker finished on time, which was a first!', 'gold': 'sarcastic'}, # Edge case: sarcasm disguised as positive observation
    {'text': "This could have been better, but it wasn't terrible.", 'gold': 'neutral'} # Edge case: vague/mixed, leans neutral
]

df = pd.DataFrame(data)
df

,text,gold
0,Loved the AI Residency session. Clear examples...,positive
1,One of the best AI workshops I have attended.,positive
2,"The mentor's explanation was crystal clear, an...",positive
3,Excellent content and delivery. Highly recomme...,positive
4,Too fast. Hard to follow the LLM and RAG parts.,negative
5,Audio issues made it difficult to concentrate.,negative
6,The pace was too slow for experienced develope...,negative
7,Disappointed with the lack of in-depth technic...,negative
8,Wow amazing… another it depends answer. Super ...,sarcastic
9,"Sure, production deployment is that easy. Tota...",sarcastic


## Step 4: Prompt Engineering Template

In [14]:
ALLOWED_LABELS = ['positive', 'negative', 'sarcastic', 'neutral']

SYSTEM_PROMPT = f"""
You are an expert sentiment and tone classifier for AI education feedback.

TASK:
- Read the feedback text
- Assign exactly ONE label from: {ALLOWED_LABELS}
- Detect sarcasm even if positive words are used
- Return STRICT JSON only

OUTPUT FORMAT:
{{
  "label": "<one of the allowed labels>",
  "confidence": <float between 0 and 1>,
  "rationale": "<one sentence explanation>",
  "action": "<one concrete action for DDS Academy>"
}}
"""

## Step 5: Classification Function

In [18]:
def classify_feedback(text, temperature=0):
    response = client.responses.create(
        model='gpt-4.1-nano-2025-04-14',
        instructions=SYSTEM_PROMPT,
        input=text,
        temperature=temperature,
    )
    return json.loads(response.output_text)
    #gpt-4.1-nano-2025-04-14'
    #gpt-3.5-turbo
    #gpt-5-2025-08-07

## Step 6: Run Classification

In [19]:
results = []

for _, row in df.iterrows():
    pred = classify_feedback(row['text'], temperature=0.2)
    results.append({
        'text': row['text'],
        'gold': row['gold'],
        'predicted': pred['label'],
        'confidence': pred['confidence'],
        'action': pred['action']
    })

results_df = pd.DataFrame(results)
results_df

,text,gold,predicted,confidence,action
0,Loved the AI Residency session. Clear examples...,positive,positive,0.95,Continue incorporating practical examples in s...
1,One of the best AI workshops I have attended.,positive,positive,0.95,Thank the participant and consider featuring t...
2,"The mentor's explanation was crystal clear, an...",positive,positive,0.95,Continue providing clear explanations and prac...
3,Excellent content and delivery. Highly recomme...,positive,positive,0.95,Thank the reviewer and consider featuring this...
4,Too fast. Hard to follow the LLM and RAG parts.,negative,negative,0.85,Consider pacing the lecture more slowly and pr...
5,Audio issues made it difficult to concentrate.,negative,negative,0.90,Investigate and improve audio quality for futu...
6,The pace was too slow for experienced develope...,negative,negative,0.90,Review and adjust the course content to better...
7,Disappointed with the lack of in-depth technic...,negative,negative,0.90,Enhance the technical content to provide more ...
8,Wow amazing… another it depends answer. Super ...,sarcastic,sarcastic,0.90,Encourage clarity and specificity in responses...
9,"Sure, production deployment is that easy. Tota...",sarcastic,sarcastic,0.90,Clarify with the student about their actual ex...


## Step 7: Evaluation

In [20]:
print('Accuracy:', accuracy_score(results_df['gold'], results_df['predicted']))
print()
print(classification_report(results_df['gold'], results_df['predicted']))

Accuracy: 0.8636363636363636

              precision    recall  f1-score   support

    negative       0.80      1.00      0.89         4
     neutral       0.83      0.83      0.83         6
    positive       0.86      1.00      0.92         6
   sarcastic       1.00      0.67      0.80         6

    accuracy                           0.86        22
   macro avg       0.87      0.88      0.86        22
weighted avg       0.88      0.86      0.86        22



# Task
Create a Gradio interface for the `classify_feedback` function, allowing users to input text and receive the predicted sentiment label, confidence, rationale, and recommended action.

## Install Gradio

### Subtask:
Add a `pip install` command to ensure Gradio is installed in the environment.


**Reasoning**:
The user wants to install Gradio. I will add a code cell with the `!pip install gradio` command.



In [21]:
get_ipython().system('pip -q install gradio')
print('✅ Gradio installed')

✅ Gradio installed


## Import Gradio

### Subtask:
Import the `gradio` library as `gr`.


**Reasoning**:
The user wants to import the `gradio` library as `gr`. I will add a code cell to perform this import.



In [22]:
import gradio as gr
print('✅ Gradio imported')

✅ Gradio imported


## Define Gradio-compatible prediction function

### Subtask:
Create a new Python function `predict_sentiment` that takes feedback text as input, calls the existing `classify_feedback` function, and formats its dictionary output into separate strings suitable for Gradio's output components (label, confidence, rationale, action).


**Reasoning**:
The user wants to define a new Python function `predict_sentiment` that will wrap the `classify_feedback` function and format its output for Gradio. This function needs to extract specific fields from the output dictionary and convert the confidence to a string.



In [23]:
def predict_sentiment(text_input):
    prediction_output = classify_feedback(text_input)
    label = prediction_output['label']
    confidence = str(prediction_output['confidence'])
    rationale = prediction_output['rationale']
    action = prediction_output['action']
    return label, confidence, rationale, action

print('✅ `predict_sentiment` function defined')


✅ `predict_sentiment` function defined


## Create and launch Gradio Interface

### Subtask:
Instantiate `gr.Interface` using the `predict_sentiment` function, define appropriate `gr.Textbox` inputs and outputs, and launch the application using `iface.launch()`.


## Summary:

### Data Analysis Key Findings

*   The Gradio library was successfully installed and imported, confirming its readiness for use in the environment.
*   A Python function named `predict_sentiment` was successfully defined. This function acts as a wrapper, taking user text input, calling the underlying `classify_feedback` function, and then extracting and formatting the 'label', 'confidence' (converted to a string), 'rationale', and 'action' into a tuple suitable for Gradio's multi-output display.

### Insights or Next Steps

*   The foundational components for the Gradio interface are now in place; the immediate next step is to instantiate the `gr.Interface` using `predict_sentiment` as the core function and define the appropriate input (`gr.Textbox`) and multiple output components to display the prediction results.
*   This structured approach of defining a wrapper function ensures that the `classify_feedback` logic remains decoupled from the presentation layer, facilitating easier testing and future modifications of either component independently.


In [24]:
iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=5, label='Enter feedback text here'),
    outputs=[
        gr.Textbox(label='Predicted Label'),
        gr.Textbox(label='Confidence'),
        gr.Textbox(label='Rationale'),
        gr.Textbox(label='Action')
    ],
    title='DDS Academy Feedback Sentiment Analyzer',
    description='Analyze sentiment of DDS Academy feedback (positive, negative, sarcastic, neutral) with rationale and action.',
    examples=[
        ['Loved the AI Residency session. Clear examples and very practical.'],
        ['Too fast. Hard to follow the LLM and RAG parts.'],
        ['Wow amazing… another it depends answer. Super helpful 🙃'],
        ['Session covered prompt engineering basics.']
    ]
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6f7f3f14f070b4f76.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
